In [ ]:
import torch
import numpy as np

from src.models import get_device, load_model, get_preprocess, get_target_layer
from src.data import load_voc2007_val
from src.explainers import (
    GradCAM,
    saliency_map,
    integrated_gradients,
    lime_explanation
)
from src.metrics import (
    binarize_attribution,
    compute_iou,
    deletion_curve,
    insertion_curve,
    compute_auc
)


In [ ]:
device = get_device()
model = load_model(device)
preprocess = get_preprocess()
target_layer = get_target_layer(model, layer_type="late")


In [ ]:
dataset = load_voc2007_val(root="./data", download=True)
print(f"Loaded VOC 2007 val set with {len(dataset)} images")


In [ ]:
# Fixed indices used for evaluation
image_indices = list(range(20))  # change only if your paper used a different count


In [ ]:
iou_results = {
    "Grad-CAM": [],
    "Saliency": [],
    "Integrated Gradients": [],
    "LIME": []
}

deletion_auc = {k: [] for k in iou_results}
insertion_auc = {k: [] for k in iou_results}


In [ ]:
for idx in image_indices:
    img_pil, gt_boxes = dataset[idx]

    # Prepare input
    input_tensor = preprocess(img_pil).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(input_tensor)
        target_class = outputs.argmax(dim=1).item()

    # --- Generate explanations ---
    gradcam = GradCAM(model, target_layer)
    cam_map = gradcam(input_tensor, target_class)

    sal_map = saliency_map(model, input_tensor, target_class)

    ig_map = integrated_gradients(
        model,
        input_tensor,
        target_class,
        baseline=None,
        steps=50
    )

    img_rgb = np.array(img_pil)
    lime_map = lime_explanation(
        model,
        preprocess,
        img_rgb,
        target_class,
        num_samples=1000
    )

    explanations = {
        "Grad-CAM": cam_map,
        "Saliency": sal_map,
        "Integrated Gradients": ig_map,
        "LIME": lime_map
    }

    # --- IoU ---
    for name, attr in explanations.items():
        binary = binarize_attribution(attr, threshold=0.5)
        iou = compute_iou(binary, gt_boxes)
        iou_results[name].append(iou)

    # --- Deletion / Insertion ---
    for name, attr in explanations.items():
        del_curve = deletion_curve(
            model, input_tensor, attr, target_class, steps=50
        )
        ins_curve = insertion_curve(
            model, input_tensor, attr, target_class, steps=50
        )

        deletion_auc[name].append(compute_auc(del_curve))
        insertion_auc[name].append(compute_auc(ins_curve))


In [ ]:
def summarize(results):
    return {
        "mean": float(np.mean(results)),
        "std": float(np.std(results))
    }

iou_summary = {k: summarize(v) for k, v in iou_results.items()}
deletion_summary = {k: summarize(v) for k, v in deletion_auc.items()}
insertion_summary = {k: summarize(v) for k, v in insertion_auc.items()}


In [ ]:
print("=== IoU ===")
for k, v in iou_summary.items():
    print(k, v)

print("\n=== Deletion AUC ===")
for k, v in deletion_summary.items():
    print(k, v)

print("\n=== Insertion AUC ===")
for k, v in insertion_summary.items():
    print(k, v)


In [ ]:
import json
from pathlib import Path

Path("results").mkdir(exist_ok=True)

with open("results/iou_summary.json", "w") as f:
    json.dump(iou_summary, f, indent=2)

with open("results/deletion_summary.json", "w") as f:
    json.dump(deletion_summary, f, indent=2)

with open("results/insertion_summary.json", "w") as f:
    json.dump(insertion_summary, f, indent=2)
